In [1]:
import os
import time
import logging
import numpy as np
import pandas as pd
from gensim import corpora, utils
from gensim.models import LdaMulticore
from gensim.models.wrappers import DtmModel

In [2]:
logging.basicConfig(level=logging.DEBUG)

# Data preprocessing

In [4]:
data = pd.read_csv("./../data/raw/training-subset.csv", converters={'np': eval})

In [5]:
data.shape

(7799, 12)

In [8]:
data = data.loc[data["year"].notnull()].copy()

In [9]:
data["year"] = data["year"].astype(int)

In [10]:
dates_cnt = data['year'].value_counts().rename_axis('year').reset_index(name='excerpt_count')
dates_cnt.sort_values('year', inplace = True, ascending = True)
time_seq = dates_cnt['excerpt_count'].to_list()

In [9]:
# #now we count the number of full texts in each years
# dates_count = data.groupby("year").agg(n_paragraphs = ("fulltext", "size"))
# dates_count = dates_count.loc[dates_count['n_paragraphs'].ne(0)].reset_index()
# dates_count["n_paragraphs"].sum()

In [11]:
# time_seq = dates_count["n_paragraphs"].to_list()
sum(time_seq)

7793

# Dictionary and BOWs

In [12]:
# create a dictionary
dictionary = corpora.Dictionary(data['np'])  
print(f'{len(dictionary)} tokens overall') 

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(524452 unique tokens: ['Algerian opposition leaders', 'Balkan leader', 'Beijing rule', 'Belgrade last week', 'Belgrade military hospital']...) from 7793 documents (total 1069049 corpus positions)


524452 tokens overall


In [14]:
# now we'll filter the tokens that appear too frequently or are too rare
dictionary.filter_extremes(no_below = 2, no_above = 0.99, keep_n=200000)
dictionary.compactify()  # make token IDs sequential

INFO:gensim.corpora.dictionary:discarding 411507 tokens: [('Balkan leader', 1), ('Beijing rule', 1), ('Belgrade last week', 1), ('Belgrade military hospital', 1), ('Bosnian Serb headquarters', 1), ('Chinese prison', 1), ('Constable Glenn Schweyer', 1), ('EUROPE Official', 1), ('Garry Styles', 1), ('Julius Rose', 1)]...
INFO:gensim.corpora.dictionary:keeping 112945 tokens which were in no less than 2 and no more than 7715 (=99.0%) documents
DEBUG:gensim.corpora.dictionary:rebuilding dictionary, shrinking gaps
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(112945 unique tokens: ['Algerian opposition leaders', 'Chinese authorities', 'Education Minister Domingo Palermo', 'European defense policy', 'French newspaper Le Monde']...)
DEBUG:gensim.corpora.dictionary:rebuilding dictionary, shrinking gaps


In [15]:
data['bows'] = data['np'].apply(dictionary.doc2bow)  # convert documents (list of tokens) to BOWs
# print(data['bows'][0]) #what we have

# DTM model training

In [ ]:
start = time.time()
model = DtmModel(
    dtm_path="C:/github/tt-weights/models/bin/dtm-win64.exe",
    corpus=data["bows"].values,
    time_slices=time_seq,
    num_topics=20,
    id2word=dictionary,
    initialize_lda=True,
    top_chain_var=0.05
)
finish = time.time()

INFO:gensim.models.wrappers.dtmmodel:serializing temporary corpus to C:\Users\hryho\AppData\Local\Temp\2023e8_train-mult.dat
INFO:gensim.corpora.bleicorpus:no word id mapping provided; initializing from corpus
INFO:gensim.corpora.bleicorpus:storing corpus in Blei's LDA-C format into C:\Users\hryho\AppData\Local\Temp\2023e8_train-mult.dat
DEBUG:smart_open.smart_open_lib:{'uri': 'C:\\Users\\hryho\\AppData\\Local\\Temp\\2023e8_train-mult.dat', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}
INFO:gensim.corpora.bleicorpus:saving vocabulary of 112945 words to C:\Users\hryho\AppData\Local\Temp\2023e8_train-mult.dat.vocab
DEBUG:smart_open.smart_open_lib:{'uri': 'C:\\Users\\hryho\\AppData\\Local\\Temp\\2023e8_train-mult.dat.vocab', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_param

In [ ]:
finish - start

In [15]:
finish - start

12163.771802186966

In [ ]:
model.save("../models/dtm-noun-phrases.model")

---

# topic weights over time

In [ ]:
model = DtmModel.load("../models/dtm-noun-phrases.model")

In [ ]:
time_seq_d = dates_cnt.set_index(dates_cnt["year"])["excerpt_count"].to_dict()

topics = {}
for idx, year in enumerate(time_seq_d.keys()):
    doc_topic, topic_term, doc_lengths, term_frequency, vocab = model.dtm_vis(time=idx,corpus=data['bows'])
    topics[year] = doc_topic

In [ ]:
topics.keys()

In [ ]:
len(topics[2010])

In [ ]:
len(topics[2010][0])

In [31]:
# first_doc_index = 0
# for year, time in zip(time_seq_d.keys(), time_seq):
#     last_doc_index = first_doc_index + time
#     topics[f"{year}_seq"] = topics[year][first_doc_index:last_doc_index]
#     first_doc_index = first_doc_index + year_slice

In [ ]:
doc_topic_matrix = pd.DataFrame() # create a placeholder df
for k, v in topics.items(): #iterate over keys and values in our dictionary with matrices
    x = pd.DataFrame.from_records(v) #create a df from the current slice matrix
    x['year_pub'] = k #create a column for year and assign the key value to it
    doc_topic_matrix = pd.concat([doc_topic_matrix, x]) #now append to our placeholder df
#let's see how it looks
doc_topic_matrix

In [ ]:
doc_topic_matrix.columns = [f"topic_{c}" if isinstance(c, int) else c for c in doc_topic_matrix.columns]

In [ ]:
doc_topic_matrix

In [ ]:
topics_over_time = pd.concat([data, doc_topic_matrix.reset_index(drop = True)], axis = 1)
topics_over_time.drop(columns = ['filename', 'year_pub'], inplace = True)

# topics_over_time['year'] = pd.to_datetime(topics_over_time['year'], format = '%Y')
topics_over_time.head()

In [ ]:
topics_over_time = pd.melt(topics_over_time, id_vars = ['author', 'database', 'doi', 
                                                    'fulltext', 'np','place','pubtitle', 'title', 'url', 
                                                    'year', 'bows'], 
            value_vars = ['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 
                          'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 
                          'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19', ],
            var_name = 'topic_num',
            value_name = 'topic_weight')
topics_over_time.head(3)

In [ ]:
years = [*topics.keys()]

In [ ]:
term_topics_by_time = {}
for t, year in enumerate(years):
    # since we have 20 topics - range(20)
    topics = []
    for n in range(20): 
        current_topic = model.show_topic(
            topicid=n,
            time=t,
            topn=15 # top 15 most salient terms
        )
        topics.append(
            # round probability of each word to three values 1.111
            [(np.around(prob, 3), word) for prob, word in current_topic]
        )
    term_topics_by_time[year] = topics

In [ ]:
topic_term_matrix = pd.DataFrame(term_topics_by_time) # create a df from the dictionary 
topic_term_matrix.head(5)

In [ ]:
topic_term_table = topic_term_matrix.T # we need to turn it
topic_term_table['year'] = topic_term_table.index.astype(int) # create an index (now indexed by years)
topic_term_table.reset_index(inplace = True, drop = True)
topic_term_table.head(3)

In [ ]:
topic_term_table.columns = [f"topic_{c}" if isinstance(c, int) else c for c in topic_term_table.columns]

In [ ]:
topic_term_table = pd.melt(topic_term_table, id_vars = 'year', 
                           value_vars = ['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 
                                         'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 
                                         'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14', 
                                         'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19'], 
                           var_name = 'topic_num', value_name = 'terms')

In [ ]:
topic_term_table

In [ ]:
topics_over_time = topics_over_time.merge(topic_term_table, how = 'left', on = ['year', 'topic_num'])

topics_over_time.head(3)

In [ ]:
topics_over_time.drop(columns = ['bows', 'doi'], inplace = True)

In [ ]:
topics_over_time = topics_over_time.loc[topics_over_time["np"].notnull()].reset_index(drop=True).copy()

In [ ]:
def simplify(l):
    return [f"{str(prob)} {word}" for prob, word in l]

In [ ]:
topics_over_time["terms"] = topics_over_time["terms"].apply(simplify)

In [ ]:
topics_over_time.to_json(
    "../data/processed/noun-phrases-topics_over_time.json", 
    lines=True,
    orient="records", 
    force_ascii=False, 
    date_format="iso"
)

# term weights over time

In [ ]:
topics = []
for t, year in enumerate(years):
    for n in range(20):
        current_topic = model.show_topic(
            topicid=n,
            time=t,
            topn=15 # top 15 most salient terms
        )
        topics.extend(
            [list(term) + [year, n] for term in current_topic]
        )

In [ ]:
terms_by_time = pd.DataFrame(topics, columns = ['weight', 'term', 'year', 'topic n'])
terms_by_time.head()

In [ ]:
terms_by_time.to_csv("../data/processed/noun-phrases-terms_over_time.csv", index=False)

In [ ]:
terms_by_time.groupby(["year", "topic n"]).sum()